# Notebook for parser development

In [1]:
## not for parser. for notebook only 

## CX: allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Loading data

Current approach: loading all files into 1 pandas dataframe (currently (4721, 21)). Then I can...
1. check the duplicates situation (key columns vs all columns) and raise errors if need be
2. remove duplicates before generating documents
3. Do some tasks column-wise over all the data, rather than while iterating over rows

If I did the generator approach (load files 1 by 1, 1 row at a time), I'd have to modify how I do things:
1. Don't do this check/raise errors. But try to mitigate potential "duplicate" issues: 
  * Sort all delimited strings
  * Use a hash of all column values (when they're all strings) for `_id`. Want rows with all the same values to produce the same hash
2. Either leave to BioThings toolset to remove duplicates, or could use a set of `_id` hashes so far to check/not create duplicate docs.
3. Do the tasks on single rows/chunks (pandas [read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#pandas.read_csv) has an iterator for rows/chunks! see iterator/chunksize parameters)


Notes:
* There are a few existing parsers that use `pandas` to load the entire raw data file at once: https://github.com/search?q=repo%3Abiothings%2Fpending.api%20pandas&type=code
* But there are other existing parsers that use `csv` to load the file **one row at a time** (generator): https://github.com/search?q=repo%3Abiothings%2Fpending.api+csv+reader&type=code

In [2]:
## put into parser: already done
import pathlib
import pandas as pd

## don't put in parser. Just for this notebook
import glob
from pprint import pprint

## unsure on putting into parser: more for notebook viewing/debugging...
pd.options.display.max_columns = None

In [3]:
## put into parser (format): DONE

base_file_path = pathlib.Path.home().joinpath("Desktop", "EBIgene2pheno_files", "From_FTP")

## pathlib's Path.glob produces a generator
## using list works to check if paths matching pattern were actually found or not
all_file_paths = list(base_file_path.glob("*.csv.gz"))
all_file_paths

## force all columns to be str type, except "date of last review" (it'll be datetime)
df = pd.concat((pd.read_csv(f, dtype=str, parse_dates=["date of last review"]) for f in all_file_paths), ignore_index=True)

## make column names snake-case - usable with itertuples later
df.columns = df.columns.str.replace(" ", "_")

df.shape
df.head()
df.info(memory_usage="deep")

[PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/CardiacG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/SkeletalG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/DDG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/SkinG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/Hearing_lossG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/CancerG2P_2025-02-28.csv.gz'),
 PosixPath('/Users/colleenxu/Desktop/EBIgene2pheno_files/From_FTP/EyeG2P_2025-02-28.csv.gz')]

(4714, 21)

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review
0,G2P00124,KCNE1,176261,6240,ISK; JLNS2; LQT5; MINK,KCNE1-related Jervell and Lange-Nielsen syndrome,612347,NaN,biallelic_autosomal,potential secondary finding,strong,altered gene product structure,missense_variant; inframe_deletion; stop_gaine...,undetermined,inferred,NaN,HP:0000407; HP:0001657; HP:0000007; HP:0001279,30461122,DD; Cardiac,KCNE1-related JLNS is due to altered gene prod...,2024-04-05 12:05:01+00:00
1,G2P00841,PTPN11,176876,9644,BPTP3; NS1; PTP2C; SH-PTP2; SHP-2; SHP2,PTPN11-related Noonan syndrome with multiple l...,151100,NaN,monoallelic_autosomal,NaN,definitive,altered gene product structure,missense_variant; inframe_deletion; inframe_in...,undetermined,inferred,NaN,HP:0000325; HP:0002996; HP:0000957; HP:0001709...,27484170; 26377839; 25917897; 25884655; 248207...,DD; Skin; Cardiac,Expert review done on 12/01/2022; Noonan syndr...,2025-01-21 14:56:43+00:00
2,G2P03247,DSC2,125645,3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:36:09+00:00
3,G2P03248,DSC2,125645,3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,biallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:35:19+00:00
4,G2P03249,DSG2,125671,3049,CDHF5,DSG2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012434,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; missense_variant; stop_gaine...,undetermined,inferred,NaN,NaN,21636032; 33831308; 33917638; 34400560; 240707...,Cardiac,Expert review done on 05/01/2022; DSG2-related...,2024-03-20 09:40:18+00:00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4714 entries, 0 to 4713
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   g2p_id                              4714 non-null   object             
 1   gene_symbol                         4714 non-null   object             
 2   gene_mim                            4712 non-null   object             
 3   hgnc_id                             4714 non-null   object             
 4   previous_gene_symbols               4241 non-null   object             
 5   disease_name                        4714 non-null   object             
 6   disease_mim                         3574 non-null   object             
 7   disease_MONDO                       638 non-null    object             
 8   allelic_requirement                 4714 non-null   object             
 9   cross_cutting_modifier              629 n

In [4]:
glob.glob("*.csv.gz")
base_file_path.glob("*.csv.gz")

[]

<generator object Path.glob at 0x107772df0>

## Checking, removing duplicates

In [6]:
## put into parser (format): DONE

## This is a data-quality check to make sure drop_duplicates actually removes all duplicates. 
## Based on exploring the data, the column subset below should uniquely define one 
##   record's data/row.
## If the de-deplicated data using this column set == de-duplicated data using the whole
##   dataset, then everything is fine and the parser can proceed with de-duplication.
## Else, the data needs to be explored and the parser probably needs adjustments.
## Many column values are delimited strings, and my concern is that these values could
##   differ (only in list order) for the "same data" in different files.

n_duplicates_column_combo = df[df.duplicated(subset=["g2p_id", "gene_symbol", "disease_name", "allelic_requirement", 
                                "molecular_mechanism"], keep=False)].shape

n_duplicates_all_columns = df[df.duplicated(keep=False)].shape

## for testing
# n_duplicates_all_columns = (1, 1)


if n_duplicates_column_combo != n_duplicates_all_columns: 
    raise AssertionError("The data format has changed, and record de-duplication may not work as-expected. " \
                          "Double-check the data and what columns uniquely define one record")

In [7]:
## put into parser (format): DONE

## drop duplicates
df.drop_duplicates(inplace=True, ignore_index=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3647 entries, 0 to 3646
Data columns (total 21 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   g2p_id                              3647 non-null   object             
 1   gene_symbol                         3647 non-null   object             
 2   gene_mim                            3645 non-null   object             
 3   hgnc_id                             3647 non-null   object             
 4   previous_gene_symbols               3277 non-null   object             
 5   disease_name                        3647 non-null   object             
 6   disease_mim                         2570 non-null   object             
 7   disease_MONDO                       561 non-null    object             
 8   allelic_requirement                 3647 non-null   object             
 9   cross_cutting_modifier              451 n

## Column-level transforms

In [9]:
df_diseasemim = df.copy()

## done to preserve NA
df_diseasemim["disease_mim"] = [i if pd.isna(i) \
                                else "OMIM:" + i if i.isnumeric() \
                                else i \
                                for i in df_diseasemim["disease_mim"]]

df_diseasemim["disease_mim"] = df_diseasemim["disease_mim"].str.replace("Orphanet", "orphanet")

In [10]:
df_diseasemim[df_diseasemim["disease_mim"].str.contains("OMIM:", na=False)].shape

df_diseasemim[df_diseasemim["disease_mim"].str.contains("orphanet:", na=False)].shape

## add up row count. If == num non-null in info above, you're good 
## right now 2570 == 2570, so good

(2561, 21)

(9, 21)

In [11]:
## put into parser (format): DONE

## COLUMN-LEVEL TRANSFORMS

## adding Translator/biolink prefixes to IDs
df["gene_mim"] = "OMIM:" + df["gene_mim"]
df["hgnc_id"] = "HGNC:" + df["hgnc_id"]
df["disease_mim"] = df["disease_mim"].str.replace("Orphanet", "orphanet")
## done to preserve NA
df["disease_mim"] = [i if pd.isna(i) \
                     else "OMIM:" + i if i.isnumeric() \
                     else i \
                     for i in df["disease_mim"]]

## strip whitespace
df["disease_name"] = df["disease_name"].str.strip()
df["comments"] = df["comments"].str.strip()

## create new columns
## UI really wants resource website urls like this. May need to adjust over time as website changes
df["g2p_record_url"] = "https://www.ebi.ac.uk/gene2phenotype/lgd/" +  df["g2p_id"]

## replace panel keywords with full names shown on G2P website for single record
## keeping "Hearing loss" as-is, changing all other values
df["panel"] = df["panel"].str.replace("DD", "Developmental disorders")
df["panel"] = df["panel"].str.replace("Cancer", "Cancer disorders")
df["panel"] = df["panel"].str.replace("Cardiac", "Cardiac disorders")
df["panel"] = df["panel"].str.replace("Eye", "Eye disorders")
df["panel"] = df["panel"].str.replace("Skeletal", "Skeletal disorders")
df["panel"] = df["panel"].str.replace("Skin", "Skin disorders")

In [12]:
## checking on column-level transforms

df.head()
# df["g2p record url"].unique()[0:100]

# df[df["disease mim"].str.contains("orphanet", na=False)]  ## 9 rows, so that's correct
# df[df["panel"].str.contains("Hearing", na=False)]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url
0,G2P00124,KCNE1,OMIM:176261,HGNC:6240,ISK; JLNS2; LQT5; MINK,KCNE1-related Jervell and Lange-Nielsen syndrome,OMIM:612347,NaN,biallelic_autosomal,potential secondary finding,strong,altered gene product structure,missense_variant; inframe_deletion; stop_gaine...,undetermined,inferred,NaN,HP:0000407; HP:0001657; HP:0000007; HP:0001279,30461122,Developmental disorders; Cardiac disorders,KCNE1-related JLNS is due to altered gene prod...,2024-04-05 12:05:01+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00124
1,G2P00841,PTPN11,OMIM:176876,HGNC:9644,BPTP3; NS1; PTP2C; SH-PTP2; SHP-2; SHP2,PTPN11-related Noonan syndrome with multiple l...,OMIM:151100,NaN,monoallelic_autosomal,NaN,definitive,altered gene product structure,missense_variant; inframe_deletion; inframe_in...,undetermined,inferred,NaN,HP:0000325; HP:0002996; HP:0000957; HP:0001709...,27484170; 26377839; 25917897; 25884655; 248207...,Developmental disorders; Skin disorders; Cardi...,Expert review done on 12/01/2022; Noonan syndr...,2025-01-21 14:56:43+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P00841
2,G2P03247,DSC2,OMIM:125645,HGNC:3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac disorders,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:36:09+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03247
3,G2P03248,DSC2,OMIM:125645,HGNC:3036,CDHF2; DSC3,DSC2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012506,biallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; splice_region_variant; misse...,undetermined,inferred,NaN,NaN,31028357; 23911551; 21636032; 33831308; 263105...,Cardiac disorders,Expert review done on 05/01/2022; DSC2-related...,2024-03-20 09:35:19+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03248
4,G2P03249,DSG2,OMIM:125671,HGNC:3049,CDHF5,DSG2-related arrhythmogenic right ventricular ...,NaN,MONDO:0012434,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,inframe_deletion; missense_variant; stop_gaine...,undetermined,inferred,NaN,NaN,21636032; 33831308; 33917638; 34400560; 240707...,Cardiac disorders,Expert review done on 05/01/2022; DSG2-related...,2024-03-20 09:40:18+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03249


## Generating documents

In [ ]:
for row in df.itertuples(index=False):
    if pd.notna(row.previous_gene_symbols):
        [i.strip() for i in row.previous_gene_symbols.split(";")]
        break

In [13]:
## put into parser (format) -> DONE. 
##   don't save in array, yield each document instead

## GENERATING DOCS, saving in array
documents = []

## using itertuples because it's faster, preserves datatypes
for row in df.itertuples(index=False):
    ## simple assignments
    document = {
        "_id": row.g2p_id,
        "subject": {
            "hgnc_symbol": row.gene_symbol,
            "hgnc": row.hgnc_id,
            "type": "Gene"
        },
        "association": {
            "g2p_record_id": row.g2p_id,
            "g2p_record_url": row.g2p_record_url,
            "allelic_requirement": row.allelic_requirement,
            "confidence": row.confidence,
            "molecular_mechanism": row.molecular_mechanism,
            "molecular_mechanism_categorisation": row.molecular_mechanism_categorisation,
            "g2p_panels": [i.strip() for i in row.panel.split(";")],
            "date_of_last_review": str(row.date_of_last_review)
        },
        "object": {
            "name": row.disease_name,
            "type": "Disease"
        }
    }    
    ## only create field if value is not NA
    ##   if value is NA, list comprehension with split won't work
    ## Gene
    if pd.notna(row.gene_mim):
        document["subject"]["omim"] = row.gene_mim
    if pd.notna(row.previous_gene_symbols):
        document["subject"]["previous_gene_symbols"] = \
        [i.strip() for i in row.previous_gene_symbols.split(";")]
        
    ## Association
    if pd.notna(row.cross_cutting_modifier):
        document["association"]["cross_cutting_modifiers"] = [i.strip() for i in row.cross_cutting_modifier.split(";")]
    if pd.notna(row.variant_consequence):
        document["association"]["variant_consequences"] = [i.strip() for i in row.variant_consequence.split(";")]
    if pd.notna(row.variant_types):
        document["association"]["variant_types"] = [i.strip() for i in row.variant_types.split(";")]
    ## uses diff delimiters, could do more parsing
    if pd.notna(row.molecular_mechanism_evidence):
        document["association"]["molecular_mechanism_evidence"] = [i.strip() for i in row.molecular_mechanism_evidence.split("&")]
    if pd.notna(row.phenotypes):
        document["association"]["phenotypes"] = [i.strip() for i in row.phenotypes.split(";")]
    if pd.notna(row.publications):
        document["association"]["pmids"] = [i.strip() for i in row.publications.split(";")]
    if pd.notna(row.comments):
        document["association"]["curator_comments"] = row.comments
     
    ## Disease
    ## disease_mim: create field depending on whether OMIM or orphanet    
    if pd.notna(row.disease_mim):
        if row.disease_mim.startswith("orphanet"):
            document["object"]["orphanet"] = row.disease_mim
        elif row.disease_mim.startswith("OMIM"):
            document["object"]["omim"] = row.disease_mim
    if pd.notna(row.disease_MONDO):
        document["object"]["mondo"] = row.disease_MONDO
    
    documents.append(document)

## Checking documents

In [36]:
df[df["panel"].str.contains("Hearing", na=False)]

## look for single values
# df[~ df["publications"].str.contains(";", na=True)]


# df[df["publications"].isna()]
# df[df["cross_cutting_modifier"].notna()]

,g2p_id,gene_symbol,gene_mim,hgnc_id,previous_gene_symbols,disease_name,disease_mim,disease_MONDO,allelic_requirement,cross_cutting_modifier,confidence,variant_consequence,variant_types,molecular_mechanism,molecular_mechanism_categorisation,molecular_mechanism_evidence,phenotypes,publications,panel,comments,date_of_last_review,g2p_record_url
3159,G2P03582,MYO6,OMIM:600970,HGNC:7605,DFNA22; DFNB37; KIAA0389,MYO6-related nonsyndromic genetic hearing loss,NaN,NaN,biallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,missense_variant; stop_gained; frameshift_vari...,undetermined,inferred,NaN,NaN,18348273; 23485424; 25999546; 12687499; 24105371,Hearing loss,NaN,2024-11-28 14:52:17+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03582
3160,G2P03583,MYO6,OMIM:600970,HGNC:7605,DFNA22; DFNB37; KIAA0389,MYO6-related nonsyndromic genetic hearing loss,NaN,NaN,monoallelic_autosomal,NaN,definitive,decreased gene product level; altered gene pro...,missense_variant; stop_gained; frameshift_vari...,undetermined,inferred,NaN,NaN,18348273; 23485424; 25999546; 24105371,Hearing loss,NaN,2024-11-28 14:47:17+00:00,https://www.ebi.ac.uk/gene2phenotype/lgd/G2P03583


In [ ]:
pprint(documents[3642])

# documents[416]

## Parser notes

Fine to use raise/assert in parser (raise is technically better programming behavior: https://realpython.com/python-assert-statement/#understanding-common-pitfalls-of-assert)


My notes on parser:
* adding prefixes to gene/disease IDs is good for pre-NodeNorming steps
* keeping diff gene/disease ID namespaces as separate fields right now is good for current BTE/x-bte-annotation system


My notes on syntax:
* use `yield` when you want to "return" within a "for loop" (return only happen once, then exit for-loop/function execution)
  * that's what it's used in main execution, when you're iterating over csv rows to generate documents
* use `yield from {function}` to get the data from a generator (created by `yield` being used the function)

## Pre-NodeNorming